In [ ]:
##Importations

import pandas as pd
import uuid
import time
import os
import copy
import datetime

In [ ]:
#Constantes et Utilitaires
CL = {"terrains à bâtir":"AB","terrains d’agrément":"AG","bois":"B","futaies feuillues":"BF","futaies mixtes":"BM",
"oseraies":"BO","peupleraies":"BP","futaies résineuses":"BR","taillis sous futaie":"BS","taillis simples":"BT",
"carrières":"CA","chemin de fer":"CH","eaux":"E","jardins":"J","landes":"L","landes boisées":"LB","prés":"P",
"pâtures":"PA","pacages":"PC","prés d’embouche":"PE","herbages":"PH","prés plantes":"PP","sols":"S","terres":"T",
"terres plantées":"TP","vergers":"VE","vignes":"VI","AB":"terrains à bâtir","AG":"terrains d’agrément","B":"bois",
"BF":"futaies feuillues","BM":"futaies mixtes","BO":"oseraies","BP":"peupleraies","BR":"futaies résineuses",
"BS":"taillis sous futaie","BT":"taillis simples","CA":"carrières","CH":"chemin de fer","E":"eaux","J":"jardins",
"L":"landes","LB":"landes boisées","P":"prés","PA":"pâtures","PC":"pacages","PE":"prés d’embouche","PH":"herbages",
"PP":"prés plantes","S":"sols","T":"terres","TP":"terres plantées","VE":"vergers","VI":"vignes","":None, "nan":None}
DISC = {"CA":"Autre", "carrières":"Autre","CH":"Autre", "chemin de fer":"Autre","E":"Autre", "eaux":"Autre",
"P":"Nature", "prés":"Nature","PA":"Nature", "pâtures":"Nature","PC":"Nature", "pacages":"Nature","PE":"Nature",
"prés d’embouche":"Nature","PH":"Nature", "herbages":"Nature","PP":"Nature", "prés plantes":"Nature","T":"Nature", 
"terres":"Nature","LB":"Nature", "landes boisées":"Nature","J":"Nature", "jardins":"Nature","L":"Nature", "landes":"Nature",
"B":"Nature", "bois":"Nature","BF":"Nature", "futaies feuillues":"Nature","BM":"Nature", "futaies mixtes":"Nature",
"BO":"Nature", "oseraies":"Nature","BP":"Nature", "peupleraies":"Nature","BR":"Nature", "futaies résineuses":"Nature",
"BS":"Nature", "taillis sous futaie":"Nature","BT":"Nature", "taillis simples":"Nature",
"S":"Terrain", "sols":"Terrain","AB":"Terrain", "terrains à bâtir":"Terrain","AG":"Terrain", "terrains d’agrément":"Terrain",
"TP":"Agri", "terres plantées":"Agri","VE":"Agri", "vergers":"Agri","VI":"Agri", "vignes":"Agri","":None, "nan":None}
def trimestre(string_date):
    y = str(string_date)[:4]
    tri = int(string_date.split("-")[1])
    trimestre = "-t1" if tri<4 else "-t2" if tri<7 else "-t3" if tri<10 else "-t4"
    return y+trimestre

def disc_terrain(surface_terrain):
    st = float(surface_terrain)
    return 0 if st<100 else 100 if st<200 else 200 if st<400 else 400 if st<800 else 800 if st<1500 else 1500
def multiple_terrain(dico_surf,valeur_tt):
    for k,v in dico_surf.items():
        if v==valeur_tt:
            return k
    return "Multi"


In [ ]:
#corriger format 2014-2018 optimisé (17secondes au lieu de 12h)
#corrige à partir du DF principal, les mutations <2018 pour leur donner un id unique (les fichiers ont ensuite été détruit :]
def reformate2_1418():
    df_1418 = df[df["id_mutation"].str.contains("M")]
    #tri pour comparaison ensuite
    df_1418 = df_1418.sort_values(by=['date_mutation','section','valeur_fonciere']).reset_index()
    bb = False
    for index, row in df_1418.iterrows():
        if bb:
            #comparaison pour rapprochement date_mutation,valeur fonciere, section
            if (df_1418.at[index-1,'date_mutation'],df_1418.at[index-1,'valeur_fonciere'],df_1418.at[index-1,'section'])==(df_1418.at[index,'date_mutation'],df_1418.at[index,'valeur_fonciere'],df_1418.at[index,'section']):
                df_1418.at[index,'id_mutation'] = df_1418.at[index-1,'id_mutation']
            else:
                df_1418.at[index,'id_mutation'] = uuid.uuid4()
        else:
            df_1418.at[index,'id_mutation'] = uuid.uuid4()
            bb=True
    #print(df_1418.head(30))
    return df_1418

In [ ]:
%%time
## Principe de cette partie du script
## A partir des données DVF etalab,
## Création d'un talbeau
## avec pour chaque lignes un type de bien d'une mutation
## avec pour colonnes annees-trimestre de la vente, id_parcelle, type bien (voir ci-dessous), prixm2, id_mutation, surface bâti, surface terrain

##type bien, label composé de 7 items
## - type bâti [maison, appart, local]
## - nb pieces principales
## - type terrain [agri, terrain, autre, multi]
## - surface terrain [0,<100, <200, <400, <800, <1500, >=1500]
## - nb dependance


##################
#une fois traités, les fichiers créés et utilisées ont été supprimé :]


#MAIN (pied)

#liste de fichiers dvf "departementalisés" 2014-2021 (les plus de 5 ans ont été détruit :]
dep = os.listdir("./dep")
#agreg = os.listdir("./dep_agreg").map(lambda x: x[0:2])
agreg = [x[0:2] for x in os.listdir("./dep_agreg")]
for d in dep:
    print(d)
    #if pour gérer les arrets brutaux et pas tout recommencer (coucou ceux qui ne branchent pas leur ordi sur le secteur)
    if d[0:2] not in agreg:
        print(f'process {d}')
        print(str(datetime.datetime.now()))
        #recup
        df = pd.read_csv('./dep/'+d, encoding="ansi", sep=",")
        
        #garde que les ventes
        df = df[df["nature_mutation"]=="Vente"]
        
        #add colonnes utiles
        df['section'] = df['id_parcelle'].map(lambda x: x[:-4])
        df['disc_sol'] = df["code_nature_culture"].map(lambda x: DISC[str(x)])
        
        #liste sections
        #sections = df['section'].unique()
        
        #reformate 1418 (fournit des id-mutation)
        df_sans = df.query("not id_mutation.str.contains('M')")
        df = pd.concat([df.query("not id_mutation.str.contains('M')"),reformate2_1418()],ignore_index=True)
        
        
        
        ##Process opti (~1 min par fichier)
        
        #Agrège toutes les lignes d'une mutations en nb ligne de biens différents
        
        
        #tri le dataframe
        df = df.sort_values(by=["id_mutation", "code_type_local"],na_position="last",ignore_index="True")
        taille_df = len(df)
        template = {
            "surf" : {
                "Autre" : 0,
                "Nature" : 0,
                "Terrain" : 0,
                "Agri" : 0
            },
            "loge" : {
                "maison" : [],   #item = [nb_p, surface]
                "appart" : [],
                "local"  : [],
            },
            "dependance":0,
            "nouveau":True,
            "prix":0,
            "date_mut": None,
            "mutation":None,
        #    "adresse":None,
            "id_parcelle":None
        }

        #copie pour ne pas passer par référence
        temp = copy.deepcopy(template)
        resultat_template = {"annee-t":[],"parcelle":[],"type_bien":[],"prixm2":[],"mutation":[],"surfaceB":[],"surfaceT":[]}
        #resultat = pd.DataFrame.from_dict(resultat_template)
        #initialise la variable de résultat
        res = copy.deepcopy(resultat_template)
        
        #
        #compartimentation du code, (pour la lisibilité lol)
        #
        
        #Permet de passer de l'objet cumulatif à des lignes agrégeant une mutation
        def temp_to_line(t):
            #res = resultat_template
            #nb bâti
            nb_loge = sum([len(x) for x in t["loge"].values()])
            #somme des surfaces terrain, (dénominateur)
            surf_terr = sum(t["surf"].values())
            #somme des surfaces bâti (dénominateur)
            surf_bati = sum([x[1] for x in t['loge']["maison"]])+sum([x[1] for x in t["loge"]["appart"]])+sum([x[1] for x in t["loge"]["local"]])
            
            #si que du terrain
            if nb_loge==0:
                for k,v in t["surf"].items():
                    if v>0:
                        res["annee-t"].append(trimestre(t["date_mut"]))
                        res["parcelle"].append(t["id_parcelle"])
                        res["type_bien"].append("§".join(["","",str(k),str(disc_terrain(v)),str(t["dependance"]) if t["dependance"]<3 else "3"]))
                        res["prixm2"].append(str((float(t["prix"])/surf_terr)))
                        res["mutation"].append(t["mutation"])
                        res["surfaceB"].append("0")
                        res["surfaceT"].append(str(v))
            #sinon            
            elif surf_bati>0 :
                #recuperation label du type terrain (classification)
                nom_terrain = multiple_terrain(t["surf"],surf_terr)
                #si une dépendance chacun
                depend_bool = nb_loge==t["dependance"]
                depend_a_affect = True
                nb_depend = t["dependance"]
                #?
                verif_surf_loc = [x for x in t["loge"]]
                #pour tous les types logements/locaux
                for labloc, valeurs in t["loge"].items():
                    #pour tous les items de chaque
                    for (p,s) in valeurs:
                        #vire les erreurs surface<=0
                        if s>0:
                            res['annee-t'].append(trimestre(t["date_mut"]))
                            res['parcelle'].append(str(t["id_parcelle"]))
                            #gestion des dependances
                            if depend_a_affect:
                                if depend_bool:
                                    nb_depend=1
                                else:
                                    #on met tous sur le premier
                                    depend_a_affect=False
                            else:
                                nb_depend = 0
                            #création du label typant le logement
                            res["type_bien"].append("§".join([labloc,
                                    str(p)+"p" if p<6 else "6p",
                                    nom_terrain,
                                    str(disc_terrain(surf_terr/nb_loge)),
                                    str(nb_depend) if nb_depend<3 else "3"
                                ]))
                            res["prixm2"].append(float(t["prix"])/surf_bati)
                            res["mutation"].append(str(t["mutation"]))
                            res["surfaceB"].append(s)
                            res["surfaceT"].append(surf_terr/nb_loge)
                            #resultat.append(res,ignore_index=True)
                            #pd.concat([resultat,pd.DataFrame.from_dict(res)],ignore_index=True)
        
        #recuperation du label à partir de l'index
        def code_to_labloge(i):
            return ["maison","appart","dep","local"][int(str(i)[0])-1]
        
        #pour une même mutation, retourne un objet représentant les lignes d'une mutation
        # 1er param une ligne
        # 2eme l'objet cumulatif
        #retourne l'objet cumulatif
        def init_row_to_temp(rowt, templat):
            res = templat
            
            #rôti
            #si il y a du sol, on cumule la surface
            if rowt.disc_sol:
                res["surf"][rowt.disc_sol] += rowt.surface_terrain if not pd.isna(rowt.surface_terrain) else 0
            #si il y a du bâti
            if not pd.isna(rowt.code_type_local):
                #si dépendance alors cumule du nb dépendance
                if int(rowt.code_type_local)==3:
                    res["dependance"] +=1
                #sinon ajout du batiment type_bati:[nb_piece, surface]
                else:
                    res["loge"][code_to_labloge(rowt.code_type_local)].append([rowt.nombre_pieces_principales,rowt.surface_reelle_bati])
            
            #si premiere insertion de la mutation, initialisation de variable commune à ladite mutation
            if res["nouveau"]:
            #if True:
                res["prix"] = rowt.valeur_fonciere
                res["id_parcelle"] = rowt.id_parcelle
                res["date_mut"] = rowt.date_mutation
                res["mutation"] = rowt.id_mutation
                res["nouveau"] = False
            return res

        
        #main
        
        #fait défiler le df trié
        for indexi, rowi in df.iterrows():
            if indexi==0:
                #par précaution même si...
                temp = init_row_to_temp(rowi, copy.deepcopy(template))
                #si il s'agit de la même mutation
            elif df.at[indexi-1,'id_mutation'] == rowi.id_mutation:        
                temp = init_row_to_temp(rowi, temp)
            #sinon
            else :
                #gestion des mutations
                temp_to_line(temp)
                #reinitialisation de la nouvelle premiere ligne de la nouvelle mutation
                temp = init_row_to_temp(rowi, copy.deepcopy(template))

        #temps de transposition complétement negligeable (0.05s)
        #temps 
        pd.DataFrame.from_dict(res).to_csv('./dep_agreg/'+d[0:2]+'_agreg.csv', header=True, index=False, encoding="ansi")



In [ ]:
#Principe du script
#création d'un tableau avec 
#pour chaque ligne une section
#pour chaque colonne (autre que la colonne id_section) un trimestre (de 2014 à 2022)

# au niveau de chaque section
# pour chaque type (label créés ci-dessus) de foncier
# si comparable alors prise en compte de l'evolution lissée chaque trimestres concernées
# pondération de l'evolution de chaque trimestre par les evoutions exactes




#creation du template de resultat, une ligne par section, pour chaque section un nb colonne = au nb trimestres 2014-2022
lt = [x for x in range((2014-2014)*4,(2023-2014)*4)]
res = {x:[] for x in lt}
res_exact = {x:[] for x in lt}
#df = df[:200]

#repertoire initial
dep = os.listdir("./dep_agreg")
#repertoire resultat
rela = [x[0:2] for x in os.listdir("./dep_rela")]

for d in dep:
    print(d)
    #gestion des arrets brutaux et historique
    if d[0:2] not in rela:
        print(f'process {d}')
        print(str(datetime.datetime.now()))
        
        #recup
        df = pd.read_csv('./dep_agreg/'+d, encoding="ansi", sep=",")
        df.columns = ["annee-t","parcelle","type_bien","prixm2","mutation","surfaceB","surfaceT"]
        df['section'] = df['parcelle'].map(lambda x: x[:-4])
        #garde que necessaire + tri 
        df = df.drop_duplicates(subset=["annee-t","parcelle","type_bien","prixm2","mutation"]).sort_values(by=["section","type_bien","annee-t"],ignore_index=True)

        
        #init variable resultat avec le template 
        #pour chaque section, une partie de comparaison exacte, une partie de comparaison normale
        resultats = {x:{"res_exact":copy.deepcopy(res_exact),"res":copy.deepcopy(res)} for x in df['section'].unique()}
        tailledf = len(df)
        tempo = []

        for ip, rp in df.iterrows():
            #print suivi
            if ip%10000==0:
                print(str(ip/len(df))+" - "+str(datetime.datetime.now()))
            
            #si même type bien que précédent
            if ip!=0 and rp["type_bien"]== df.at[ip-1,'type_bien']:
                #print(ip)
                #mmb = parcelle[(parcelle["section"]==rp["section"]) and (parcelle["type_bien"]==rp["type_bien"])]
                #mmb = df.query("section==@rp.section and type_bien==@rp.type_bien")
                
                #comparaison avec tous les biens du même type (au sein de la même section bien entendu), et si comparable alors ajout de l'évolution lissée au trimestre
                #rmm les elements dans tempo
                #comparés avec rp, l'élément du for courant
                #rp est ensuite ajouté à tempo
                for rmm in tempo:
                    #si pas même prix (donc pas meme bien)
                    if rmm['prixm2'] != rp['prixm2']:
                        #recuperation des trimestres, prix, et "ordonnancement" du plus antèrieur 
                        a_n0, a_t0 = rmm["annee-t"].split("-t")
                        p0 = float(rmm["prixm2"])
                        a_n1, a_t1 = rp["annee-t"].split("-t")
                        p1 = float(rp["prixm2"])
                        ts0, ts1 = (int(a_n0)-2014)*4+int(a_t0), (int(a_n1)-2014)*4+int(a_t1)
                        distance = abs(ts0-ts1)
                        ts0,ts1,p0,p1  = (ts0,ts1,p0,p1) if ts0<ts1 else (ts1,ts0,p1,p0)
                        
                        #booleen comparaison exacte
                        b_exact = (rp["surfaceB"],rp["surfaceT"],rmm["parcelle"])==(rp["surfaceB"],rp["surfaceT"],rp["parcelle"])
                        #init de l'interval acceptée (min/max +/- 5% par trimestre pour un bien) permet de virer les valeurs abérantes et les incomparables
                        inter = 0.1 if b_exact else 0.05
                        limin, limax = p0*(pow(1-inter,distance)),p0*(pow(1+inter,distance))
                        
                        #si dans l'interval
                        if limin<p1<limax:
                            evol_t = (p1/p0)**(1/(ts1-ts0))
                            #si compraison exacte
                            if b_exact:
                                #ajout du coefficient directeur pour les trimestres de la section pour EXACT
                                for t in range(ts0,ts1+1): resultats[rp['section']]["res_exact"][t].append(evol_t)
                            else:
                                #ajout du coefficient directeur pour les trimestres de la section
                                for t in range(ts0,ts1+1): resultats[rp['section']]["res"][t].append(evol_t)
                #stack tempo
                tempo.append({"prixm2":rp["prixm2"],"annee-t":rp["annee-t"],"surfaceB":rp["surfaceB"],"surfaceT":rp["surfaceT"],"parcelle":rp["parcelle"]})
            
            #si bien different, reinitialisation du tempo avec la nouvelle premiere valeur du nouveau type de bien
            else:
                tempo = [{"prixm2":rp["prixm2"],"annee-t":rp["annee-t"],"surfaceB":rp["surfaceB"],"surfaceT":rp["surfaceT"],"parcelle":rp["parcelle"]}]
        print(str(datetime.datetime.now()))
        
        #moyenne des coéfficients directeurs 
        #avec s'il y a lieu
        #pondération de moitié (genre une moyenne) des comparaisons normales par les comparaisons exactes
        for s in resultats.keys():
            resultats[s]["res_fin"] = {}
            #pour tous les trimestres, si des comparaisons exactes pour le trimestre alors pondération, sinon
            for t in lt:
                moyres = None
                if len(resultats[s]["res"][t])>0:
                    moyres = sum(resultats[s]["res"][t])/len(resultats[s]["res"][t])
                    if len(resultats[s]["res_exact"][t])>0:
                        moyres = (moyres+sum(resultats[s]["res_exact"][t]))/(1+len(resultats[s]["res_exact"][t]))
                elif len(resultats[s]["res_exact"][t])>0:
                    moyres = sum(resultats[s]["res_exact"][t])/len(resultats[s]["res_exact"][t])
                resultats[s]["res_fin"][t] = moyres
            resultats[s] = resultats[s]["res_fin"]
        pd.DataFrame.from_dict(resultats,orient="index").to_csv('./dep_rela/'+d[0:2]+'_rela.csv', header=True, index=True, encoding="ansi")
        #resultats
print("FIN")

In [ ]:
%%time
##
##Rassemblement des fichiers départementalisés en un fichier national
##

dep = [x for x in os.listdir("./dep_rela") if x[-4:]==".csv"]
liste = []
for d in dep:
    liste.append(pd.read_csv('./dep_rela/'+d, encoding="ansi", sep=","))
df = pd.concat(liste)
len(df)
%%time
df.rename(columns = {"Unnamed: 0":'section'}, inplace = True)
df["Code INSEE"] = df['section'].map(lambda x: x[:5])
#df = pd.merge(df, corr, how="left", on="Code INSEE")
#df

#si quelqu'un voit ça et peut me dire comment encoder dynamiquement de l'ansi à de l'unicode je suis preneur
df.to_csv('./dvf_evolution_section.csv', header=True, index=False, encoding="ansi")